In [98]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from nltk.corpus import stopwords
from tqdm import tqdm
import numpy as np

In [99]:
stopwords = stopwords.words('french')
import Stemmer
stemmer = Stemmer.Stemmer('french')

In [100]:
def stem(x):
    x = x.lower()
    x = x.replace("l'",'').replace("d'",'')
    string = ' '.join(
        stemmer.stemWords(
            [x for x in x.split(' ') if x not in stopwords])
        )
    string = string.replace(',','')
    return string

tqdm.pandas()

In [101]:
train = pd.read_csv('data/raw/naf_activite.csv', sep='|', index_col=0)
train.dropna(inplace=True)
test = pd.read_csv('data/raw/test.csv', index_col=0)

In [102]:
train['ACTIVITE'] = train['ACTIVITE'].progress_apply(stem)
test['text'] = test['text'].progress_apply(stem)

100%|██████████| 292/292 [00:00<00:00, 21637.69it/s]


In [103]:
mapping = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/7bb2184b-88cb-4c6c-a408-5a0081816dcd', sep=',')

# statistics above have no in-line sepators - remove from mapping
mapping['id_5'] = mapping['id_5'].str.replace('.','')

# create series to merge
#naf5_naf2_naf1 = mapping.set_index('')[['id_2', 'id_1']]
train = train.merge(
    mapping[['id_2', 'id_5']],
    left_on='NAF_CODE', right_on='id_5',
    how='inner')

train = train[['ACTIVITE','id_2']]

/var/folders/cz/nm0sr8dn1vd806c3239_r7k00000gn/T/ipykernel_92345/393509108.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  mapping['id_5'] = mapping['id_5'].str.replace('.','')


In [104]:
train = train.iloc[:250_000]

vectorizer = TfidfVectorizer(max_df=.7, max_features=300)
#pca = PCA(n_components=0.95)

test_text = vectorizer.fit_transform(raw_documents=test['text']).toarray()
train_text = vectorizer.transform(raw_documents=train['ACTIVITE']).toarray()

In [105]:
from sklearn.ensemble import ExtraTreesClassifier
classifier = ExtraTreesClassifier(n_jobs=-1, verbose=10)

In [106]:
classifier.fit(train_text, train['id_2'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100building tree 5 of 100
building tree 6 of 100building tree 7 of 100


building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.6s


building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   11.1s


...

building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100


[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  1.0min


building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done  96 out of 100 | elapsed:  1.2min remaining:    3.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.2min finished


ExtraTreesClassifier(n_jobs=-1, verbose=10)

In [107]:
preds  = pd.DataFrame(classifier.predict_proba(test_text), columns=classifier.classes_)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  96 out of 100 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


In [122]:
priors = pd.read_csv('data/priors.csv')[['naf2','employees']].set_index('naf2')
priors = np.log(priors)
priors = priors.loc[~priors.index.isin([84,85,86])]

/Users/hjalmarheld/miniforge3/envs/ds2/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [125]:
multiplied_preds = {}
for i in preds.index:
    pred_series = pd.Series(np.log(preds.loc[i]))
    #pred_series = pred_series.add(priors.T, fill_value=0).T
    pred_str = ' '.join([str(i) for i in pred_series.sort_values(ascending=False).index[:10]])
    multiplied_preds[i] = pred_str

/Users/hjalmarheld/miniforge3/envs/ds2/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [126]:
multiplied_preds

{0: '68 41 70 38 85',
 1: '70 68 85 38 41',
 2: '70 68 85 41 38',
 3: '68 70 41 85 38',
 4: '68 41 70 38 85',
 5: '70 85 68 38 41',
 6: '68 70 85 41 38',
 7: '68 70 41 85 38',
 8: '70 68 85 38 41',
 9: '70 68 85 41 38',
 10: '70 85 68 38 41',
 11: '41 68 38 70 85',
 12: '68 70 85 41 38',
 13: '68 41 70 85 38',
 14: '70 85 68 38 41',
 15: '70 85 68 38 41',
 16: '70 85 68 38 41',
 17: '70 85 68 41 38',
 18: '68 70 41 85 38',
 19: '68 38 70 85 41',
 20: '68 70 41 85 38',
 21: '68 41 70 38 85',
 22: '68 38 41 85 70',
 23: '68 70 38 41 85',
 24: '70 85 68 38 41',
 25: '70 68 38 85 41',
 26: '70 85 68 38 41',
 27: '38 85 70 68 41',
 28: '70 68 85 41 38',
 29: '68 85 70 41 38',
 30: '68 85 41 70 38',
 31: '70 85 68 38 41',
 32: '68 41 70 38 85',
 33: '68 70 85 41 38',
 34: '70 85 68 38 41',
 35: '68 70 85 38 41',
 36: '70 68 85 38 41',
 37: '38 70 68 85 41',
 38: '70 68 41 38 85',
 39: '70 68 85 38 41',
 40: '68 70 38 85 41',
 41: '68 70 38 85 41',
 42: '70 68 38 41 85',
 43: '68 41 70 85 38'

In [112]:
pred_df = pd.DataFrame(pd.Series(multiplied_preds), columns=['Predicted'])

In [113]:
pred_df

,Predicted
0,68 41 85 70 38 1 2 3 5 6
1,70 85 68 38 41 1 2 3 5 6
2,70 85 68 41 38 1 2 3 5 6
3,68 85 70 41 38 1 2 3 5 6
4,68 41 70 38 85 1 2 3 5 6
...,...
287,85 68 70 41 38 1 2 3 5 6
288,85 70 68 41 38 1 2 3 5 6
289,85 68 70 41 38 1 2 3 5 6
290,70 85 68 41 38 1 2 3 5 6


In [96]:
pred_df.index = pred_df.index.rename('NewsId')

In [97]:
pred_df.to_csv('tfidf.csv')

In [41]:
priors

0     0.010226
1     0.000839
2     0.000736
3     0.000000
4     0.000011
        ...   
80    0.000667
81    0.008639
82    0.012449
83    0.001415
84    0.009142
Name: employees, Length: 85, dtype: float64

In [35]:
preds

,1,2,3,6,7,8,9,10,11,12,...,86,87,88,90,91,92,93,94,95,96
0,0.005000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.010000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000172,0.000000,0.0,0.000000,0.000000,0.000000,0.005000
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,0.002646,0.000049,0.000092,0.0,0.000002,0.000003,0.0,0.042725,0.00001,0.0,...,0.000184,0.000005,0.000044,0.020083,0.000002,0.0,0.010213,0.000002,0.000046,0.030960
3,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.010000
4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.002500,0.00000,0.0,...,0.000000,0.000000,0.030000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.040000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,0.000714,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.010526,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.001053
288,0.040000,0.003333,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.002000,0.000000,0.000000,0.000000,0.000000,0.0,0.010000,0.000000,0.030000,0.000000
289,0.020000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
290,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.020000,0.000000,0.020000,0.010000
